# cifar10_dataloading

> How to load cifar10 datasets, for SSL and supervised learning
 

In [ ]:
#| default_exp cifar10_dataloading

In [ ]:
#| hide
from nbdev.showdoc import *
from fastcore.test import *

In [ ]:
#| export
import torch
from fastai.vision.all import *
# from self_supervised.augmentations import *
# from self_supervised.layers import *
from base_rbt.helper import *

CIFAR10

In [ ]:
#| export

def get_bt_cifar10_train_dls(bs,device,num_workers=12):
  
    return ImageDataLoaders.from_lists(path, fnames_train, labels_train,bs=bs, item_tfms=[Resize(size=size)], #batch_tfms=[ToTensor(), IntToFloatTensor()],
                                  valid_pct=0.0,num_workers=num_workers,device=device,seed=seed
                                      )


def get_supervised_cifar10_train_dls(bs,device,num_workers=12):

    return ImageDataLoaders.from_lists(path, fnames_train, labels_train,bs=bs, item_tfms=[Resize(size=size)], #batch_tfms=[ToTensor(), IntToFloatTensor()],
                                  valid_pct=0.0,num_workers=num_workers,device=device,seed=seed
                                      )

def get_supervised_cifar10_test_dls(bs,device,num_workers=12):
    return ImageDataLoaders.from_lists(path, fnames_test, labels_test,bs=bs, item_tfms=[Resize(size=size)], #batch_tfms=[ToTensor(), IntToFloatTensor()],
                                  valid_pct=0.0,num_workers=num_workers,device=device,seed=seed
                                      )
                                      

def get_supervised_cifar10_test_dls():pass

seed=42
size=32
path = untar_data(URLs.CIFAR)
fnames_train = get_image_files(path / "train") 
fnames_train.sort()
#shuffle data (in reproducible way)
seed_everything(seed=seed)
fnames_train = fnames_train.shuffle()

fnames_test = get_image_files(path / "test")

def label_func(fname):
    return fname.name.split('_')[1].strip('png').strip('.')

#labels for train,eval and test
labels_train = [label_func(fname) for fname in fnames_train]
labels_test = [label_func(fname) for fname in fnames_test]

test_eq(len(labels_train),len(fnames_train))
test_eq(len(labels_test),len(fnames_test))
test_eq(len(set(labels_train)),10)
test_eq(len(labels_train),50000)
test_eq(len(labels_test),10000)



In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()